In [5]:
!pip install pandas dask psutil kaggle --quiet


In [23]:
from google.colab import files

uploaded = files.upload()

os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)

for filename in uploaded.keys():
    os.rename(filename, os.path.expanduser("~/.kaggle/kaggle.json"))

os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), 0o600)



Saving kaggle.json to kaggle.json


In [24]:
import os, zipfile


!kaggle datasets download -d mkechinov/ecommerce-behavior-data-from-multi-category-store


with zipfile.ZipFile("ecommerce-behavior-data-from-multi-category-store.zip", "r") as z:
    z.extractall(".")

!ls -lh
print("الملفات الموجودة:", os.listdir("."))

Dataset URL: https://www.kaggle.com/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store
License(s): copyright-authors
ecommerce-behavior-data-from-multi-category-store.zip: Skipping, found more recently modified local copy (use --force to force download)
total 20G
-rw-r--r-- 1 root root 8.4G Oct 14 23:12 2019-Nov.csv
-rw-r--r-- 1 root root 5.3G Oct 14 23:13 2019-Oct.csv
-rw-r--r-- 1 root root 1.6G Oct 14 23:08 data_compressed.csv.gz
-rw-r--r-- 1 root root 4.3G Dec  9  2019 ecommerce-behavior-data-from-multi-category-store.zip
drwxr-xr-x 1 root root 4.0K Oct 13 13:43 sample_data
الملفات الموجودة: ['.config', '.ipynb_checkpoints', 'data_compressed.csv.gz', '2019-Oct.csv', '2019-Nov.csv', 'ecommerce-behavior-data-from-multi-category-store.zip', 'sample_data']


In [25]:
import pandas as pd

csv_file = "/content/2019-Oct.csv"

sample_df = pd.read_csv(csv_file, nrows=5)
print("معاينة أولية للبيانات:")
print(sample_df.head())

معاينة أولية للبيانات:
                event_time event_type  product_id          category_id  \
0  2019-10-01 00:00:00 UTC       view    44600062  2103807459595387724   
1  2019-10-01 00:00:00 UTC       view     3900821  2053013552326770905   
2  2019-10-01 00:00:01 UTC       view    17200506  2053013559792632471   
3  2019-10-01 00:00:01 UTC       view     1307067  2053013558920217191   
4  2019-10-01 00:00:04 UTC       view     1004237  2053013555631882655   

                         category_code     brand    price    user_id  \
0                                  NaN  shiseido    35.79  541312140   
1  appliances.environment.water_heater      aqua    33.20  554748717   
2           furniture.living_room.sofa       NaN   543.10  519107250   
3                   computers.notebook    lenovo   251.74  550050854   
4               electronics.smartphone     apple  1081.98  535871217   

                           user_session  
0  72d76fde-8bb3-4e00-8c23-a032dfed738c  
1  9333dfbd-b87

In [26]:
import time

start_time = time.time()

chunk_size = 100000
chunks = []

for chunk in pd.read_csv(csv_file, chunksize=chunk_size):
    chunks.append(len(chunk))

total_rows = sum(chunks)
elapsed_chunks = time.time() - start_time

print(f"عدد الصفوف الكلي: {total_rows}")
print(f"الوقت المستغرق: {elapsed_chunks:.2f} ثانية باستخدام chunksize")



عدد الصفوف الكلي: 42448764
الوقت المستغرق: 103.32 ثانية باستخدام chunksize


In [27]:
!pip install dask

import dask.dataframe as dd
import time

start_time = time.time()

df_dask = dd.read_csv(csv_file)

row_count = df_dask.shape[0].compute()

elapsed_dask = time.time() - start_time

print(f"عدد الصفوف: {row_count}")
print(f"الوقت المستغرق بـ Dask: {elapsed_dask:.2f} ثانية")


عدد الصفوف: 42448764
الوقت المستغرق بـ Dask: 106.48 ثانية


In [28]:
import gzip
import shutil
import time

start_compress = time.time()
with open(csv_file, 'rb') as f_in:
    with gzip.open('data_compressed.csv.gz', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
compress_time = time.time() - start_compress

start_read = time.time()
df_compressed = pd.read_csv('data_compressed.csv.gz', compression='gzip', nrows=50000)
read_time = time.time() - start_read

print(f" وقت الضغط: {compress_time:.2f} ثانية")
print(f" وقت القراءة بعد الضغط: {read_time:.2f} ثانية")
print(f" حجم الملف قبل وبعد الضغط:")
!ls -lh *.csv*


 وقت الضغط: 536.31 ثانية
 وقت القراءة بعد الضغط: 0.33 ثانية
 حجم الملف قبل وبعد الضغط:
-rw-r--r-- 1 root root 8.4G Oct 14 23:12 2019-Nov.csv
-rw-r--r-- 1 root root 5.3G Oct 14 23:13 2019-Oct.csv
-rw-r--r-- 1 root root 1.6G Oct 14 23:27 data_compressed.csv.gz


In [29]:
import pandas as pd

results = pd.DataFrame({
    'الطريقة': ['Pandas + chunksize', 'Dask', 'Pandas + Compression'],
    'الزمن التقريبي (ثواني)': [elapsed_chunks, elapsed_dask, read_time],
    'ملاحظات': [
        'آمنة على الذاكرة لكنها بطيئة عند المعالجة',
        'سريعة وتدعم المعالجة الموزعة',
        'توفر في المساحة لكنها تحتاج وقت ضغط أولي'
    ]
})

results


,الطريقة,الزمن التقريبي (ثواني),ملاحظات
0,Pandas + chunksize,134.088869,آمنة على الذاكرة لكنها بطيئة عند المعالجة
1,Dask,106.484279,سريعة وتدعم المعالجة الموزعة
2,Pandas + Compression,0.331341,توفر في المساحة لكنها تحتاج وقت ضغط أولي
